思考：如何通过MP（尽可能多的）得到所需要的数据？

通过之前的例子总结运行了以下的例子：

In [ ]:
from  pymatgen.ext.matproj  import MPRester#Materials Project REST API的适配器
a = MPRester("S3lw1QomLO8bTiT7")#这将初始化REST适配器，并且将API秘钥加入其中，可以从MP网站中获取到数据
all_entries = a.get_entries_in_chemsys(['CaO'])#This gets all entries belonging to the Ca-C-O system.
print(all_entries)

得到如下的结果：

In [ ]:
[ComputedEntry mp-10683 - Ca1
Energy = -1.6033
Correction = 0.0000
Parameters:
run_type = GGA
is_hubbard = False
pseudo_potential = {'functional': 'PBE', 'labels': ['Ca_sv'], 'pot_type': 'paw'}
hubbards = {}
potcar_symbols = ['PBE Ca_sv']
oxide_type = None
Data:
oxide_type = None,


得到的结果由于项目太少，排除掉这个方法，但是这也提供了用函数去调用的数据的思考方式，于是用MPRester的函数库去找找有没有相关可以符合条件的函数进行调用。

In [ ]:
class MPRester(object):
    supported_properties = ("energy", "energy_per_atom", "volume",
                            "formation_energy_per_atom", "nsites",
                            "unit_cell_formula", "pretty_formula",
                            "is_hubbard", "elements", "nelements",
                            "e_above_hull", "hubbards", "is_compatible",
                            "spacegroup", "task_ids", "band_gap", "density",
                            "icsd_id", "icsd_ids", "cif", "total_magnetization",
                            "material_id", "oxide_type", "tags", "elasticity")

    supported_task_properties = ("energy", "energy_per_atom", "volume",
                                 "formation_energy_per_atom", "nsites",
                                 "unit_cell_formula", "pretty_formula",
                                 "is_hubbard",
                                 "elements", "nelements", "e_above_hull",
                                 "hubbards",
                                 "is_compatible", "spacegroup",
                                 "band_gap", "density", "icsd_id", "cif")
    
    
    
        def __init__(self, api_key=None,
                 endpoint="https://www.materialsproject.org/rest/v2"):
        if api_key is not None:
            self.api_key = api_key
        else:
            self.api_key = SETTINGS.get("PMG_MAPI_KEY", "")
        self.preamble = endpoint
        import requests
        if sys.version_info[0] < 3:
            try:
                from pybtex import __version__
            except ImportError:
                warnings.warn("If you query for structure data encoded using MP's "
                            "Structure Notation Language (SNL) format and you use "
                            "`mp_decode=True` (the default) for MPRester queries, "
                            "you should install dependencies via "
                            "`pip install pymatgen[matproj.snl]`.")
        self.session = requests.Session()
        self.session.headers = {"x-api-key": self.api_key}
        ..............

如代码所示：supported_properties和supported_task_properties里面有很多项，猜测这就是能用MPRester所支持的属性，之后底下还有一系列的函数，函数的内容也和MPRester调用数据相关，所以从这个角度入手学习，探索是否能得到MP数据库里的数据。